In [1]:
import imp
import Data
import utils
import Predictors
imp.reload(Data)
imp.reload(utils)
imp.reload(Predictors)

from Road_Graph import *
from Predictors import *
from Data import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch
from torch import nn, autograd
import torch.nn.functional as F

In this part, we focus on a simple road map (rep as a graph below), and generate random path on it.
<img src="../img/naive_road.png" alt="Drawing" style="width: 400px;"/>

# build the road graph as above


In [2]:
graph_dict = {}
graph_dict = graph_dict.fromkeys(range(24))
edges = [(i, i+1) for i in range(5)] \
        + [(i+6, i+7) for i in range(5)]\
                        +[(i+12, i+13) for i in range(5)]\
                     + [(i+18, i+19) for i in range(5)]\
                     + [(0,6),(6,12), (12,18)]\
                     + [(pair[0]+1, pair[1]+1) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+2, pair[1]+2) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+3, pair[1]+3) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+4, pair[1]+4) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+5, pair[1]+5) for pair in [(0,6),(6,12), (12,18)]]
for i,j in edges:
    if not graph_dict[i]:
        graph_dict[i] = [j]
    else:
        graph_dict[i].append(j)
    if not graph_dict[j]:
        graph_dict[j] = [i]
    else:
        graph_dict[j].append(i)

#display(graph_dict)
graph = Road_Graph(graph_dict)

In [3]:
data = random_walk_data(graph, 10000,go_back = True)

In [5]:
mk_pd = Markov_Predictor(graph)

In [6]:
mk_pd.train(data)
nxt,prob = mk_pd.predict([0,1,2])
print(nxt, prob)

[1 3 8] [ 0.33386924  0.33386924  0.33226152]


In [7]:
mk_pd.eval(random_walk_data(graph, 1000,go_back = True))

0.61312825514869762

In [ ]:
rm_pd = Random_Predictor(graph)
rm_pd.eval(random_walk_data(graph, 1000,go_back = True))

In [ ]:
nxt

In [ ]:
F.cross_entropy
log_loss(nxt == 3, prob)

In [ ]:
def training_set(path, graph):
    N = len(graph.nodes)
    n = len(path)
    inp = torch.from_numpy(np.float32(np.array([one_hot(p, N) for p in path[:-1]])))
    tar = [int(np.where(graph.neighbors(path[i]) == path[i+1])[0]) for i in range(n - 1)]
    tar = torch.from_numpy(np.array(tar))
    return Variable(inp).contiguous(), Variable(tar).contiguous()

In [ ]:
#def train(self, data, batch_size = 50, learning_rate = 0.001, steps = 10000):                                                                                                  
data = random_walk_data(graph, 10000,go_back = True)

batch_size = 50
learning_rate = 0.001 
steps = 1000
D_in, D_hidden, D_out = len(graph.nodes), 100, 4                                                                                                                      

model = torch.nn.Sequential(                                                                                                                                               
    torch.nn.Linear(D_in, D_hidden),                                                                                                                                       
    torch.nn.ReLU(),                                                                                                                                                       
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1)
)                                                                                                                                             
loss_fn = torch.nn.CrossEntropyLoss()                                                                                                                                      
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)                                                                                                         
num_batch = len(data) // batch_size

losses = []
for t in range(1,steps+1):
    if t%100 == 0:
        print(t)
    i = t % num_batch
    data_i = data[i*batch_size:(i+1)*batch_size]
    batch_loss = Variable(torch.zeros(1), requires_grad=True)
    num_road = 0
    y_pred_list = []
    y_list = []
    for path in data_i:
        x,y = training_set(path, graph)                                                                                                                                          
        y_pred = model(x)                                                                                                                                                      
        loss = loss_fn(y_pred, y) * len(y) # TODO: truancate y_pred, Contatinate
        batch_loss = batch_loss + loss
        num_road = num_road + len(y)
    batch_loss /= num_road
    #print(batch_loss.data[0])
    losses.append(batch_loss.data[0])
    if i == 0:
        np.random.shuffle(data)
    optimizer.zero_grad()                                                                                                                                                  
    batch_loss.backward()                                                                                                                                                        
    optimizer.step()

In [ ]:
torch.sum(prob,1)

In [ ]:
def reweight(prob, cut):
    a,b = prob.size()
    for i in range(len(cut)):
        n = cut[i]
        if n < b:
            prob[i,n:] = 0
            prob[i, :n] = prob[i, :n] / torch.sum(prob[i, :n])
    return prob

def count_exit(path):
    return [len(graph.neighbors(i)) for i in path]

In [ ]:
prob = F.softmax(autograd.Variable(torch.randn(2, 3)), dim = 1)
cut = [1,2]
reweight(prob,cut)

In [ ]:
table = {0:{1:1, 2:4}, 1:{0:1,2:1}}
np.array(list(table[0])) / sum(list(table[0]))

In [ ]:
#def train(self, data, batch_size = 50, learning_rate = 0.001, steps = 10000):                                                                                                  
data = random_walk_data(graph, 10000,go_back = True)

batch_size = 50
learning_rate = 0.001 
steps = 1000
D_in, D_hidden, D_out = len(graph.nodes), 100, 4                                                                                                                      

model = torch.nn.Sequential(                                                                                                                                               
    torch.nn.Linear(D_in, D_hidden),                                                                                                                                       
    torch.nn.ReLU(),                                                                                                                                                       
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1)
)

    
loss_fn = torch.nn.CrossEntropyLoss()                                                                                                                                      
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)                                                                                                         
num_batch = len(data) // batch_size

losses = []
for t in range(1,steps+1):
    if t%100 == 0:
        print(t)
    i = t % num_batch
    data_i = data[i*batch_size:(i+1)*batch_size]
    batch_loss = Variable(torch.zeros(1), requires_grad=True)
    num_road = 0
    y_pred_list = []
    y_list = []
    for path in data_i:
        num_exit = count_exit(path)[:-1]
        x,y = training_set(path, graph)                                                                                                                                          
        y_pred = reweight(model(x),num_exit)
        y_list.append(y)
        y_pred_list.append(y_pred)
        
    batch_loss = loss_fn(torch.cat(y_pred_list), torch.cat(y_list))
        #num_road = num_road + len(y)
    #batch_loss /= num_road
    losses.append(batch_loss.data[0])
    if i == 0:
        print(batch_loss.data[0])
        np.random.shuffle(data)
    optimizer.zero_grad()                                                                                                                                                  
    batch_loss.backward()                                                                                                                                                        
    optimizer.step()

In [ ]:
plt.plot(losses)

In [ ]:
path

In [ ]:
x,y = training_set(path, graph)
path = [len(graph.neighbors(inv_one_hot(r)))  for r in x.data.numpy()]
print(path)
l = [len(graph.neighbors(r))  for r in path]
y_  = model(x)
#print(y_)

def f(y_, l):
    for i in range(len(l)):
        print(y_[i,:l[i]]) #=  y_[i,:[i]] / torch.sum(y_[i,:[i]])
        print(y_[i,l[i]:]) #= 0
f(y_,l)

In [ ]:
N_batch, D_in, D_hidden, D_out = 50, 24, 100, 4
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1),
)
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 0.002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
L = 0
n = 0
pts = {'x':[], 'y':[]}
for t in range(1,40001):
    x,y = random_training_set()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    L += sum(loss.data.numpy())
    n += len(loss.data.numpy())
    if t%1000 == 0:
        pts['x'].append(t)
        pts['y'].append(L/n)
        print(t, L/n)
        L = 0
        n = 0
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
plt.plot(pts['x'],pts['y'])

In [ ]:
path

In [ ]:
N_batch, D_in, D_hidden, D_out = 50, 24, 100, 24
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1),
)
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 0.002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
L = 0
n = 0
pts = {'x':[], 'y':[]}
for t in range(1,40001):
    x,y = random_training_set_with_road_tar()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    L += sum(loss.data.numpy())
    n += len(loss.data.numpy())
    if t%1000 == 0:
        pts['x'].append(t)
        pts['y'].append(L/n)
        print(t, L/n)
        L = 0
        n = 0
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
x,y = random_training_set_with_road_tar()
print([inv_one_hot(r) for r in x.data.numpy()])
np.concatenate([model(x)[0,:].data.numpy().reshape(24,1), 
                np.array(range(0,24)).reshape(24,1)],
               axis = 1)

In [ ]:
input
output